In [ ]:
import pandas as pd
import numpy as np

#Handling first data set
df1 = pd.read_csv('Mine.csv')

def clean_dataset1(df):
    #there is a abnormally large duration of 400
    median_duration = df[df['Duration'] <= 60]['Duration'].median() #get median of durations less than or equal to 60
    df.loc[df['Duration'] > 60, 'Duration'] = median_duration

    #the dates have unnecessary apostrophes:
    df['Date'] = df['Date'].astype(str).str.strip("'")
    #line 19 has a wrong date format.
    df['Date'] = df['Date'].replace("20231018","2023/10/18")
    #to get correct datetime objects (2023-10-18) rather than a string
    df['Date'] = pd.to_datetime(df1['Date'], errors='coerce')  #coerce errors to NaT(Not a Time)
    #since the dates are in chronological order,we can use the date before to fill in a Nat date
    df['Date'] = df['Date'].fillna(method='ffill') #forward fill
    
    #assuming pulse and calories is really essential for the dataset to have meaning
    df = df.dropna(subset=['Pulse','Calories'])
    
    # For numeric columns, fill with median (more robust than mean)
    df['Maxpulse'] = df['Maxpulse'].fillna(df['Maxpulse'].median())
    
    
    #ensure correct data types
    df = df.astype({
        'Duration': 'int',
        'Pulse': 'int',
        'Maxpulse': 'int',
        'Calories': 'float'
    })
    
    #to remove any duplicate rows
    df = df.drop_duplicates()
    
    return df

#clean the first dataset
clean_mine_df = clean_dataset1(df1)
clean_mine_df.to_csv('Cleaned_Mine.csv', index=False)
    

In [2]:
import pandas as pd

#handle second data set
df2 = pd.read_csv('Sales.csv')

def clean_dataset2(df):
    #remove rows with missing customer names and product names
    df = df.dropna(subset=['Customer Name', 'Product'])
    # Remove duplicates based on Order ID and Product
    df = df.drop_duplicates(subset=['Order ID', 'Product'])
    
    # Clean 'Order Date'
    df['Order Date'] = df['Order Date'].astype(str).str.strip("'")
    df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
    
     #Handle missing/negative value
    df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
    df['Unit Price'] = pd.to_numeric(df['Unit Price'], errors='coerce')
    df['Total Revenue'] = pd.to_numeric(df['Total Revenue'], errors='coerce')
    
    # Replace negative quantities with absolute value
    df['Quantity'] = df['Quantity'].abs()
    df['Total Revenue'] = df['Total Revenue'].abs()
    
    # Fill missing Quantity and Unit Price where Revenue is available
    df['Quantity'] = df['Quantity'].fillna(df['Total Revenue'] / df['Unit Price'])
    df['Unit Price'] = df['Unit Price'].fillna(df['Total Revenue'] / df['Quantity'])
    
    #recalculate Total Revenue
    df['Total Revenue'] = df['Quantity'] * df['Unit Price']

    
    # Ensure correct data types 
    df = df.astype({
        'Order ID': 'str',
        'Customer Name': 'str',
        'Product': 'str',
        'Quantity': 'int',
        'Unit Price': 'float',
        'Total Revenue': 'float'
    })
    
    
    return df

clean_sales_df = clean_dataset2(df2)
clean_sales_df.to_csv('Cleaned_Sales.csv', index=False)
    
    